In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams["figure.dpi"] = 150

In [ ]:
# Initialize saturation map
LONG_LEG_LENGTH = 7 # must be odd
INITIAL_HOMO_SAT = 1

sat_array_empty = np.zeros((LONG_LEG_LENGTH, (LONG_LEG_LENGTH+1)//2))


In [ ]:
# Function blocks

def construct_simulation_cell_mask(sat_arr):
    """Constructs mask of all cells that are active in the simulation
    """
    simulation_cell_mask = np.copy(sat_arr)
    
    for i in range(0, LONG_LEG_LENGTH):
        width = (i+2)//2
        simulation_cell_mask[LONG_LEG_LENGTH - i-1,:width] = np.full(width, 1)

    return simulation_cell_mask

# def construct_diffusion_mask(sim_cell_mask)



In [ ]:
# Default seed 


In [ ]:
A = np.array(
    [
        [
            np.eye(2), np.eye(2)
        ],
        [
            np.eye(2), np.eye(2)
        ]
        ]
)

In [ ]:
A[:,:,:,:]

In [ ]:
A[0,0,:,:]

In [ ]:
Arr = np.empty((10,10,10,10))

for i in range(10):
    for j in range(10):
        temp_array = np.zeros((10,10))
        temp_array[i,j] = 2

        Arr[i,j,:,:] = temp_array

In [ ]:
Arr[0,1,:,:]

In [ ]:
diffusor_submask_inner = np.array(
    [
        [0, 1/6, 1/6],
        [1/6, 0, 1/6],
        [1/6, 1/6, 0]
    ]
)

diffusor_submask_lower_ragged = np.array(
    [
        [0, 1/3],
        [1/3, 0],
        [1/3, 0]
    ]
)

diffusor_submask_upper_ragged = np.array(
    [
        [0, 1/6, 1/6],
        [1/3, 0, 0],
        [1/6, 1/6, 0]
    ]
)

diffusor_submask_flush = np.array(
    [
        [1/6, 1/3],
        [0, 1/3],
        [1/6, 0]
    ]
)

In [ ]:
def sum_NN(a, indices):
    """Sums values of nearest neighbors in a hexagonal lattice
    
    """
    # if index is legit, append
    # if index is SOMEVALUE, add to list
    # create mask for tensor

### TO-DO

- Make diffusor into a class


In [ ]:
class Diffusor:

    def __init__(self, a):
        self.a = a

    def __repr__(self):
        return f"You must be looking for '{self.a}'"

In [ ]:
test = Diffusor(2)

In [ ]:
test

In [ ]:
A = np.zeros((100,100,100,100))

In [ ]:
B = np.zeros((1000,1000))

In [ ]:
for i in range(100):
    np.tensordot(A, B)

In [ ]:
np.shape(A)

In [ ]:
A

In [ ]:
sat_array_empty[1,1] = 1

In [ ]:
np.tensordot(sat_array_empty[0:3,0:3], diffusor_submask_inner)

In [ ]:
class Converter:
    """
    A class that allows for conversion from cartesian coordinates
    to index in flattened list and vice-versa for custom hexagon-based
    simulation area.

    Attributes:
        L (int): The vertical length of simulation grid, always odd.

    Methods: FIIILLLLL OUUUTTTT

    """
    def __init__(self, L):
        self.L = L 
        self.W = int((self.L+1)/2) # Calculates horizontal length of grid
        self.total_indices = int(self.W*self.W) # Uses Gauss' formula to calculate total size (see doc)

        ##### construct list of coords #####

        cartesian_vertical_coord_list = np.ceil(2*np.sqrt(np.arange(0,self.total_indices)+1) - 1)
        cartesian_horizontal_coord_list = np.floor((cartesian_vertical_coord_list+1)/2) - np.floor(np.square((cartesian_vertical_coord_list+1)/2))

        self.coord_array = np.empty((self.total_indices, 2), dtype=int)

        for i in range(self.total_indices):
            self.coord_array[i, :] = (self.L - cartesian_vertical_coord_list[i], cartesian_horizontal_coord_list[i] + i)

        ##### construct array of indices #####

        self.index_array = np.full((self.L, self.W), np.nan)
        for i in range(self.total_indices):
            self.index_array[self.coord_array[i,0], self.coord_array[i,1]] = i
        
    def __repr__(self):
        """
        Reserved __repr__ method.

        Arguments:
            None

        Returns:
            (str): Displays total vertical and horizontal breadth of array considered.
        """
        return f"L = {self.L}; W = {self.W}"

    def coords_to_index(self, coords):
        """
        A method that converts array coordinates of a site to 
        its position in flattened array.

        Arguments:
            coords (1D array): array coordinates of a simulation site
        
        Returns:
            (int) Index in the 1D flattened representation.
        """
        return int(self.index_array[coords[0], coords[1]])

    def index_to_coords(self, index:int):
        """
        A method that converts position in flattened array to 
        its coordinates in two dimensionnal form.

        Arguments:
            index (int): index to be converted to coordinates
        
        Returns:
            (int): line of the simulation site
            (int): column of the simulation site
        """
        coords = self.coord_array[index]
        return coords[0], coords[1]



In [ ]:
gaming = Converter(1001)
gaming.coord_array
gaming.index_array